### ***Python Code***

In [2]:
# Install PuLP if not already installed
!pip install -q pulp

# Import necessary libraries
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Load the data from Excel (assuming it is stored on Google Drive)
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Colab Notebooks/organic_food_supply_chain_data.xlsx'

# Read the Excel file
df_prep_centers = pd.read_excel(file_path, sheet_name='PrepCenters')
df_stores_demand = pd.read_excel(file_path, sheet_name='StoresDemand')
df_transport_cost = pd.read_excel(file_path, sheet_name='TransportCostPrepToStore')

# Convert the data into dictionaries for easy use in the PuLP model
PrepCenters = list(df_prep_centers['PrepCenter'])
Stores = list(df_stores_demand['Store'])

TransportCostOrchardToPrep = dict(zip(df_prep_centers['PrepCenter'], df_prep_centers['TransportCostOrchardToPrep']))
PrepCost = dict(zip(df_prep_centers['PrepCenter'], df_prep_centers['PrepCost']))
Capacity = dict(zip(df_prep_centers['PrepCenter'], df_prep_centers['Capacity']))
Demand = dict(zip(df_stores_demand['Store'], df_stores_demand['Demand']))

TransportCostPrepToStore = {(row['PrepCenter'], row['Store']): row['TransportCostPrepToStore']
                            for idx, row in df_transport_cost.iterrows()}

# Initialize the model
model = LpProblem("Organic_Food_Supply_Chain_Optimization", LpMinimize)

# Decision variables
x = LpVariable.dicts("x", PrepCenters, lowBound=0, cat='Continuous')  # Orchard to prep centers
y = LpVariable.dicts("y", [(i, j) for i in PrepCenters for j in Stores], lowBound=0, cat='Continuous')  # Prep centers to stores

# Objective function: Minimize total cost
model += (
    lpSum(TransportCostOrchardToPrep[i] * x[i] + PrepCost[i] * x[i] for i in PrepCenters)
    + lpSum(TransportCostPrepToStore[(i, j)] * y[(i, j)] for i in PrepCenters for j in Stores)
)

# Constraints

# Demand constraints: Supply to each store must meet or exceed the demand
for j in Stores:
    model += lpSum(y[(i, j)] for i in PrepCenters) >= Demand[j], f"Demand_{j}"

# Capacity constraints: The amount processed at each center must not exceed its capacity
for i in PrepCenters:
    model += lpSum(y[(i, j)] for j in Stores) <= Capacity[i], f"Capacity_{i}"

# Flow balance: Apples sent from orchard to prep centers must equal the amount sent from prep centers to stores
for i in PrepCenters:
    model += x[i] == lpSum(y[(i, j)] for j in Stores), f"FlowBalance_{i}"

# Solve the model
model.solve()

# Get and print the results
print(f"Total cost is: {value(model.objective)}\n")

# Amount transported from orchard to preparation centers
for i in PrepCenters:
    print(f"Amount transported from orchard to preparation center {i}: {value(x[i])}")

# Amount transported from preparation centers to stores
for i in PrepCenters:
    for j in Stores:
        print(f"Amount transported from preparation center {i} to store {j}: {value(y[(i, j)])}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total cost is: 3040.0

Amount transported from orchard to preparation center 1: 300.0
Amount transported from orchard to preparation center 2: 500.0
Amount transported from orchard to preparation center 3: 600.0
Amount transported from preparation center 1 to store OrganicOrchard: 0.0
Amount transported from preparation center 1 to store FreshLocal: 100.0
Amount transported from preparation center 1 to store HealthyPantry: 0.0
Amount transported from preparation center 1 to store SeasonsHarvest: 200.0
Amount transported from preparation center 2 to store OrganicOrchard: 0.0
Amount transported from preparation center 2 to store FreshLocal: 100.0
Amount transported from preparation center 2 to store HealthyPantry: 400.0
Amount transported from preparation center 2 to store SeasonsHarvest: 0.0
Amount transported from preparation center 3 to store OrganicOrchard:

### ***AMPL Code***

In [3]:
# Install dependencies
!pip install -q amplpy ampltools

# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# Instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.8 MB/s eta 0:00:00
Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import pandas to read Excel data
import pandas as pd

# Define the file path
file_path = '/content/drive/MyDrive/Colab Notebooks/organic_food_supply_chain_data.xlsx'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Read the Excel file
df_prep_centers = pd.read_excel(file_path, sheet_name='PrepCenters')
df_stores_demand = pd.read_excel(file_path, sheet_name='StoresDemand')
df_transport_cost = pd.read_excel(file_path, sheet_name='TransportCostPrepToStore')

# Convert the data into dictionaries to pass into AMPL
PrepCenters = list(df_prep_centers['PrepCenter'])
Stores = list(df_stores_demand['Store'])

TransportCostOrchardToPrep = dict(zip(df_prep_centers['PrepCenter'], df_prep_centers['TransportCostOrchardToPrep']))
PrepCost = dict(zip(df_prep_centers['PrepCenter'], df_prep_centers['PrepCost']))
Capacity = dict(zip(df_prep_centers['PrepCenter'], df_prep_centers['Capacity']))
Demand = dict(zip(df_stores_demand['Store'], df_stores_demand['Demand']))

# Convert transport cost from prep centers to stores into a dictionary
TransportCostPrepToStore = {(row['PrepCenter'], row['Store']): row['TransportCostPrepToStore']
                            for idx, row in df_transport_cost.iterrows()}

In [6]:
# Define the model in AMPL
ampl.eval('''

reset;

# Define sets
set PrepCenters;
set Stores;

# Define parameters
param TransportCostOrchardToPrep {PrepCenters};
param PrepCost {PrepCenters};
param Capacity {PrepCenters};
param Demand {Stores};
param TransportCostPrepToStore {PrepCenters, Stores};

# Define decision variables
var x {i in PrepCenters} >= 0;  # Amount transported from orchard to preparation centers
var y {i in PrepCenters, j in Stores} >= 0;  # Amount transported from preparation centers to stores

# Objective: Minimize the total cost
minimize TotalCost:
    sum {i in PrepCenters} (TransportCostOrchardToPrep[i] * x[i] + PrepCost[i] * x[i])
    + sum {i in PrepCenters, j in Stores} (TransportCostPrepToStore[i,j] * y[i,j]);

# Demand constraints: Supply to each store must meet or exceed the demand
subject to DemandSatisfaction {j in Stores}:
    sum {i in PrepCenters} y[i,j] >= Demand[j];

# Capacity constraints: Each preparation center cannot exceed its processing capacity
subject to CapacityConstraint {i in PrepCenters}:
    sum {j in Stores} y[i,j] <= Capacity[i];

# Flow balance: Apples sent from orchard to prep centers must equal the amount sent from prep centers to stores
subject to FlowBalance {i in PrepCenters}:
    x[i] = sum {j in Stores} y[i,j];

''')


In [7]:
# Pass the data to the AMPL model
ampl.set['PrepCenters'] = PrepCenters
ampl.set['Stores'] = Stores
ampl.param['TransportCostOrchardToPrep'] = TransportCostOrchardToPrep
ampl.param['PrepCost'] = PrepCost
ampl.param['Capacity'] = Capacity
ampl.param['Demand'] = Demand
ampl.param['TransportCostPrepToStore'] = TransportCostPrepToStore

In [8]:
# Use ampl.expand to confirm AMPL model syntax is working
ampl.eval('expand;')

# Set solver option and solve the model
ampl.setOption('solver', 'cbc')

# Solve the model
ampl.solve()

minimize TotalCost:
	0.6*x[1] + 1.2*x[2] + 1.8*x[3] + 0.8*y[1,'OrganicOrchard'] + 
	1.1*y[1,'FreshLocal'] + 0.7*y[1,'HealthyPantry'] + 
	1.4*y[1,'SeasonsHarvest'] + 1.2*y[2,'OrganicOrchard'] + 
	1.1*y[2,'FreshLocal'] + 0.5*y[2,'HealthyPantry'] + 
	1.4*y[2,'SeasonsHarvest'] + 0.2*y[3,'OrganicOrchard'] + 
	1.4*y[3,'FreshLocal'] + 1.3*y[3,'HealthyPantry'] + 
	1.7*y[3,'SeasonsHarvest'];

subject to DemandSatisfaction['OrganicOrchard']:
	y[1,'OrganicOrchard'] + y[2,'OrganicOrchard'] + y[3,'OrganicOrchard']
	 >= 300;

subject to DemandSatisfaction['FreshLocal']:
	y[1,'FreshLocal'] + y[2,'FreshLocal'] + y[3,'FreshLocal'] >= 500;

subject to DemandSatisfaction['HealthyPantry']:
	y[1,'HealthyPantry'] + y[2,'HealthyPantry'] + y[3,'HealthyPantry'] >= 
	400;

subject to DemandSatisfaction['SeasonsHarvest']:
	y[1,'SeasonsHarvest'] + y[2,'SeasonsHarvest'] + y[3,'SeasonsHarvest']
	 >= 200;

subject to CapacityConstraint[1]:
	y[1,'OrganicOrchard'] + y[1,'FreshLocal'] + y[1,'HealthyPantry'] + 
	y[1,'Se

In [9]:
# Get and print the results
obj = ampl.get_objective('TotalCost')
print("\nTotal cost is: ", obj.get().value(), "\n")

# Print the decision variable results
print("Amount transported from orchard to preparation centers (x):")
ampl.display('x')

print("Amount transported from preparation centers to stores (y):")
ampl.display('y')


Total cost is:  3040.0 

Amount transported from orchard to preparation centers (x):
x [*] :=
1  300
2  500
3  600
;

Amount transported from preparation centers to stores (y):
y :=
1 FreshLocal       100
1 HealthyPantry      0
1 OrganicOrchard     0
1 SeasonsHarvest   200
2 FreshLocal       100
2 HealthyPantry    400
2 OrganicOrchard     0
2 SeasonsHarvest     0
3 FreshLocal       300
3 HealthyPantry      0
3 OrganicOrchard   300
3 SeasonsHarvest     0
;

